In [17]:
import pandas as pd

# Load the CSV
df = pd.read_csv('donnees_vol_multiple.csv')  # use your actual filename if different
df['is_attack'] = 1
# List of columns to keep
columns_to_keep = [
    'mission_id', 'time', 'real_x', 'real_y', 'real_z','target_x', 'target_y', 'target_z',
    'vx', 'vy', 'vz', 'ax', 'ay', 'az',
    'roll', 'pitch', 'yaw',
    'wind_x', 'wind_y', 'wind_z',
    'erreur_m', 'velocity',
    'rain', 'fog','snow' ,'is_attack'      

]

# Filter the DataFrame
filtered_df = df[columns_to_keep]

# Save the result
filtered_df.to_csv('donnees_nettoyee_bon.csv', index=False)

print(" Saved cleaned CSV with selected columns to 'donnees_nettoyee.csv'")

 Saved cleaned CSV with selected columns to 'donnees_nettoyee.csv'


In [18]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('donnees_nettoyee_bon.csv')

# Final list of valid flight rows
output_rows = []

# Track current block of rows
current_block = []
current_id = None

# Counter for assigning new clean mission_ids
new_flight_id = 1

# Go through rows one by one
for _, row in df.iterrows():
    row_id = row['mission_id']

    if current_id is None:
        current_id = row_id
        current_block.append(row)
    elif row_id == current_id:
        current_block.append(row)
    else:
        # ID changed, time to process the current block
        if len(current_block) == 21:
            for r in current_block:
                r['mission_id'] = new_flight_id
                output_rows.append(r)
            new_flight_id += 1

        # Start a new block
        current_block = [row]
        current_id = row_id

# Handle the last block
if len(current_block) == 21:
    for r in current_block:
        r['mission_id'] = new_flight_id
        output_rows.append(r)

# Convert to DataFrame
output_df = pd.DataFrame(output_rows)

# Save to CSV
output_df.to_csv('donnees_vol_multiple_filtered_bon.csv', index=False)

print("✅ Saved cleaned flight data with new mission IDs to 'cleaned_flight_data.csv'")

✅ Saved cleaned flight data with new mission IDs to 'cleaned_flight_data.csv'


In [19]:
import pandas as pd

# Charger les données
df = pd.read_csv("donnees_vol_multiple_filtered_bon.csv")

# Liste des vols valides
vols_valides = []

# Vérification des conditions sur chaque vol
for mission_id, groupe in df.groupby("mission_id"):
    groupe_sorted = groupe.sort_values("time")
    premiers_wp = groupe_sorted.head(7)

    # Vérifier les deux conditions
    condition_erreur_7points = (premiers_wp["erreur_m"] > 2.5).any()
    moyenne_vol = groupe_sorted["erreur_m"].mean()
    condition_moyenne = moyenne_vol >= 13

    # Appliquer le OU : on supprime le vol si l'une des deux conditions est vraie
    if condition_erreur_7points or condition_moyenne:
        continue
    
    vols_valides.append(mission_id)

# Filtrer le DataFrame avec les vols valides
df_final = df[df["mission_id"].isin(vols_valides)]

# Vérifier que les colonnes nécessaires sont présentes
required_cols = ['erreur_m', 'real_x', 'real_y', 'real_z', 'target_x', 'target_y', 'target_z']
if all(col in df_final.columns for col in required_cols):
    condition = df_final['erreur_m'] > 2.5
    # Appliquer la correction
    df_final.loc[condition, ['real_x', 'real_y', 'real_z']] = df_final.loc[condition, ['target_x', 'target_y', 'target_z']].values
    print(f"\n✅ Correction appliquée à {condition.sum()} points avec erreur > 2.5m")
    print(f"✅ Shape des données corrigées: {df_final.loc[condition, ['real_x', 'real_y', 'real_z']].shape}")
else:
    missing_cols = [col for col in required_cols if col not in df_final.columns]
    print(f"\n⚠️ Impossible d'appliquer la correction - Colonnes manquantes: {missing_cols}")

# Supprimer les colonnes target_x, target_y, target_z si elles existent
cols_a_supprimer = ['target_x', 'target_y', 'target_z']
df_final = df_final.drop(columns=[col for col in cols_a_supprimer if col in df_final.columns])

# Sauvegarder dans un nouveau fichier CSV
df_final.to_csv("donnees_vols_valides_B.csv", index=False)

# Affichage des statistiques finales
moyenne_finale = df_final["erreur_m"].mean()
print(f"\n📊 Moyenne finale des erreurs : {moyenne_finale:.2f} mètres")
print(f"📦 Nombre de vols restants : {len(vols_valides)}")
print("💾 Fichier 'donnees_vols_valides_B_1.csv' enregistré avec succès.")



✅ Correction appliquée à 224 points avec erreur > 2.5m
✅ Shape des données corrigées: (224, 3)

📊 Moyenne finale des erreurs : 1.78 mètres
📦 Nombre de vols restants : 71
💾 Fichier 'donnees_vols_valides_B_1.csv' enregistré avec succès.


C:\Users\Amatek\AppData\Local\Temp\ipykernel_18392\3327738729.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.loc[condition, ['real_x', 'real_y', 'real_z']] = df_final.loc[condition, ['target_x', 'target_y', 'target_z']].values


In [20]:
print(df.isna().any().any())
colonnes_avec_nan = df.columns[df.isna().any()].tolist()

# Afficher les colonnes concernées
print("Colonnes contenant des NaN :")
print(colonnes_avec_nan)

# Afficher le nombre de NaN par colonne
print("\nNombre de NaN par colonne :")
print(df[colonnes_avec_nan].isna().sum())

False
Colonnes contenant des NaN :
[]

Nombre de NaN par colonne :
Series([], dtype: float64)
